In [ ]:
#!jupyter nbconvert --to=python DO2_WOA_Comparison_python3.ipynb

In [ ]:
class DO2comp:
    '''
    class DO2comp
    
    -use layerwise = True to compare a set of depth
    -use depth_limit to specify maximum depth for mean-over-depth comparison
    '''
    def __init__(self,runname,resultpath,savepath,meshpath,ncpath,first_year,last_year,
                 WOAvar='o_an',
                 mapproj='pc',
                 cmap = 'viridis',
                 savefig=False,
                 layerwise=False,depth_array=[],
                 depth_limit = 100):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.meshpath = meshpath
        self.ncpath = ncpath
        self.fyear = first_year
        self.lyear = last_year
        self.WOAvar = WOAvar
        self.mapproj = mapproj
        self.cmap = cmap
        self.savefig = savefig
        self.layerwise = layerwise
        self.depth_array = depth_array
        self.depth_limit = depth_limit

        import matplotlib.pyplot as plt
        import numpy as np
        #from netCDF4 import Dataset
        #from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        #import pickle

        import pyfesom2 as pf

        from load_interp_WOA_python3 import WOAdata
        from plot_Taylor_normalized import plt_Taylor_norm

        # load FESOM data -------------------------------------------------------------------------------------
        mesh       = pf.load_mesh(meshpath)
        years = np.arange(self.fyear, self.lyear+1,1)

        # check variables
        #NCfesom = self.resultpath + '/DO2.'+self.runname+'.'+str(self.fyear)+'.nc'
        #!ncdump -h $NCfesom

        labelfesom = 'FESOM DIN {0}-{1}'.format(self.fyear,self.lyear)
        unitfesom = 'DO2 [mmol O2 m$^{-3}$]' 
        
        # load data -------------------------------------------------------------------------------------
        DO2fesom = pf.get_data(resultpath, "O2", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=True)


        # load WOA data  -------------------------------------------------------------------------------------
        WOA_input = WOAdata(self.runname,self.resultpath,self.meshpath,self.ncpath,self.WOAvar, get_overview=False)
        O2_int = WOA_input.woa_int     
        
        labelwoa = 'World Ocean Atlas DO2'
        unitwoa = 'DO2 [mmol O2 m$^{-3}$]' #??? ## o_an:units = "micromoles_per_kilogram"

        # apply sea mask to WOA as in FESOM ----------------------------------------------------------------------------------
        # assumption: there is no ocean where value in FESOM == 0
        O2_int_ma = np.copy(O2_int)
        O2_int_ma[DO2fesom == 0] = 0

        # plot WOA and FESOM ----------------------------------------------------------------------------------
        if(self.layerwise):
            if(self.depth_array == []):
                depth_array = (0,50,200,1000,2000,4000)

            for d in depth_array:
                # get mesh index closest to desired depth
                i = pf.ind_for_depth(d,mesh) 
                # get midlevel depth
                plot_depth = str((mesh.zlev[i]+mesh.zlev[i+1])/2)

                if(True):
                    print('\nInput depth = {0}, plotting at level depth = {1} m\nFESOM min = {2:4.4f}, max = {3:4.4f}\nWOA min = {4:4.4f}, max = {5:4.4f}'.format(
                        d,plot_depth,
                        np.nanmin(DO2fesom[:,i]),np.nanmax(DO2fesom[:,i]),
                        np.nanmin(O2_int_ma[:,i]),np.nanmax(O2_int_ma[:,i])))

                pf.plot(mesh, [O2_int_ma[:,i],DO2fesom[:,i], O2_int_ma[:,i]-DO2fesom[:,i]], 
                        rowscol= (1,3),
                        levels = (0,45,40),
                        units=unitwoa + '\n at depth = {0} m'.format(plot_depth), 
                        mapproj=self.mapproj, # robinson projection takes more time!
                        cmap = self.cmap,
                        titles=[labelwoa, labelfesom, 'WOA - FESOM'],figsize = (20,20)
                       )

            # statistics  -------------------------------------------------------------------------------------
            # preparation of datasets
            if np.isnan(np.min(O2_int)): print('WARNING: The interpolated WOA field contains NaNs at depth')
            if np.isnan(np.min(DO2fesom)): print('WARNING: The interpolated FESOM field contains NaNs at depth')


            for d in depth_array:
                # get mesh index closest to desired depth
                i = pf.ind_for_depth(d,mesh) 
                # get midlevel depth
                plot_depth = str((mesh.zlev[i]+mesh.zlev[i+1])/2)
                
                title = 'Taylor Diagram for DO$_2$ at {0} m'.format(plot_depth)
                plt_Taylor_norm(O2_int[i,:],DO2fesom[i,:],mask=True,title=title)

                if False:
                    # get statistics only from ocean gridpoints (same mask assumption as above)
                    ind_stat = np.where(DO2fesom[i,:] != 0)

                    taylor_stats1 = sm.taylor_statistics(O2_int[i,ind_stat],DO2fesom[i,ind_stat])
                    sdev = np.array([taylor_stats1['sdev'][0], taylor_stats1['sdev'][1]])
                    crmsd = np.array([taylor_stats1['crmsd'][0], taylor_stats1['crmsd'][1]])
                    ccoef = np.array([taylor_stats1['ccoef'][0], taylor_stats1['ccoef'][1]])

                    label = ['Observation', 'Model']# at {0} m'.format(plot_depth)]

                    fig = plt.figure(figsize=(7,7), facecolor='w', edgecolor='k')
                    sm.taylor_diagram(sdev,crmsd,ccoef, styleOBS = '-', colOBS = 'r', markerobs = 'o',
                                          titleOBS = 'observation', markerLabel = label,
                                          markerLabelColor = 'c',
                                          markerColor = 'c', markerLegend = 'on',
                                          tickRMS = range(0,5,1), tickRMSangle = 135.0,
                                          colRMS = 'm', styleRMS = ':', widthRMS = 2.0,
                                          titleRMS = 'off', tickSTD = range(0,10,2 ),
                                          axismax = 10.0, 
                                          colSTD = 'b', styleSTD = '-.',
                                          widthSTD = 1.0, titleSTD = 'on',
                                          colCOR = 'k', styleCOR = '--', widthCOR = 1.0,
                                          titleCOR = 'on')
                    plt.title('Taylor Diagram for DO2 at {0} m'.format(plot_depth), loc='right')
        
            if(self.savefig==True): print('\n***\n***Too many figures to export...\n***')
        
        # mean over depth  -------------------------------------------------------------------------------------
        else:
            
            # get level depth index closest to depth_limit
            i_depth_limit = pf.ind_for_depth(self.depth_limit, mesh)
            
            # corresponding level depth
            depth_limit_level = - mesh.zlev[i_depth_limit]
            
            # corresponding layer depth (where concentrations are defined):
            depth_limit_layer = - WOA_input.layer_depths[i_depth_limit]
            
            # print overview of chosen depth
            print('DO2 as mean over depth \nwith max layer depth = {0} \n(level depth = {1}m, mesh index {2})'.format(
                depth_limit_layer,depth_limit_level,i_depth_limit))
            
            # mean over depth
            DO2fesom_mean = np.mean(DO2fesom[:,:i_depth_limit], axis = 1)
            O2_int_ma_mean = np.mean(O2_int_ma[:,:i_depth_limit], axis = 1)

            print('\nFESOM min = {0:4.4f}, max = {1:4.4f}\nWOA min = {2:4.4f}, max = {3:4.4f}'.format(
                    np.nanmin(DO2fesom_mean),np.nanmax(DO2fesom_mean),
                    np.nanmin(O2_int_ma_mean),np.nanmax(O2_int_ma_mean)))
    
            fig_data = pf.plot(mesh, [O2_int_ma_mean,DO2fesom_mean], 
                    rowscol= (1,2),
                    levels = (0,280,71),
                    units=unitwoa, 
                    mapproj=self.mapproj, # robinson projection takes more time!
                    cmap = self.cmap,
                    titles=['World Ocean Atlas: Interpolated DO2 for initialization', 
                            'Mean over depth (max={2}m)\nFESOM Dissolved O$_2$ ({0}-{1})'.format(
                                first_year,last_year,depth_limit_layer),
                            #'Initialization fields minus FESOM'
                           ],
                    figsize = (20,20)
                   )
        
            # fig export  -------------------------------------------------------------------------------------
            if(self.savefig==True):
                plt.savefig(self.savepath+self.runname+'_'+'DO2_WOA'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight')
            plt.show(block=False)
            
            # plot differences seperatly for better overview
            fig_data = pf.plot(mesh, [O2_int_ma_mean-DO2fesom_mean], 
                    rowscol= (1,1),
                    levels = (-50,50,101),
                    units=unitwoa, 
                    mapproj=self.mapproj, # robinson projection takes more time!
                    cmap = 'RdBu',
                    titles=['Initialization fields minus FESOM'],
                    figsize = (10,10)
                   )
      
                
            # statistics  -------------------------------------------------------------------------------------            
            # preparation of datasets
            if np.isnan(np.min(O2_int_ma_mean)): print('WARNING: The interpolated WOA field contains NaNs at depth')
            if np.isnan(np.min(DO2fesom_mean)): print('WARNING: The interpolated FESOM field contains NaNs at depth')

            title = 'Taylor Diagram for DO$_2$ \n(mean over depth, max = {0}m)'.format(depth_limit_layer)
            plt_Taylor_norm(O2_int_ma_mean,DO2fesom_mean,mask=True,title=title)

            # Taylor fig export  
            if(self.savefig==True):                
                plt.savefig(self.savepath+self.runname+'_'+'DO2_WOA_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight')
            plt.show(block=False)  
        



In [ ]:
### TESTING ###
if __name__ == "__main__":
    
    # run specification -------------------------------------------------------------------------------------
    runname      =  'fesom'
    resultpath = '/work/ollie/mozeisin/results/f2r1.2/' + 'mo20'
    savepath = '/home/ollie/mozeisin/evaluation/mo_files/'

    meshpath = '/work/ollie/mozeisin/mesh/mesh_fesom2.0/core2_meanz'

    # period of analysis ------------------------------------------------------------------------------------

    first_year = 1965
    last_year  = 1966

    # WOA ------------------------------------------------------------------------------------    
    ncpath                = '/work/ollie/projects/MarESys/evaluation/woa18_all_o00_01.nc'
    #!ncdump -h $ncpath

    # now test:
    test = DO2comp(runname,resultpath,savepath,meshpath,ncpath,first_year,last_year,
                   cmap = 'viridis',
                   mapproj = 'pc',
                   layerwise=False,
                   depth_limit = 200,
                   savefig=False)